In [1]:
from sentence_transformers import SentenceTransformer

bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

In [2]:
import datasets

dataset = datasets.load_dataset('ms_marco', 'v2.1', split='train[:100000]').shuffle(seed=42)
dataset = dataset.train_test_split(test_size=0.1, seed=42)

Found cached dataset ms_marco (/home/ubuntu/.cache/huggingface/datasets/ms_marco/v2.1/2.1.0/b6a62715fa5219aea5275dd3556601004cd63945cb63e36e022f77bb3cbbca84)
Loading cached shuffled indices for dataset at /home/ubuntu/.cache/huggingface/datasets/ms_marco/v2.1/2.1.0/b6a62715fa5219aea5275dd3556601004cd63945cb63e36e022f77bb3cbbca84/cache-89575f570aa58446.arrow
Loading cached split indices for dataset at /home/ubuntu/.cache/huggingface/datasets/ms_marco/v2.1/2.1.0/b6a62715fa5219aea5275dd3556601004cd63945cb63e36e022f77bb3cbbca84/cache-82afc0a1dfe05f54.arrow and /home/ubuntu/.cache/huggingface/datasets/ms_marco/v2.1/2.1.0/b6a62715fa5219aea5275dd3556601004cd63945cb63e36e022f77bb3cbbca84/cache-839d92ed9c39c390.arrow


In [3]:
import torch
import torch.nn as nn
class Model(nn.Module):
    def __init__(self, encoding_dim):
        super().__init__()
        self.hidden_dim = 2 * encoding_dim
        self.feed_forward = nn.Sequential(
            nn.Linear(self.hidden_dim, self.hidden_dim * 4),
            nn.ReLU(),
            nn.Linear(self.hidden_dim * 4, self.hidden_dim * 4),
            nn.ReLU(),
            nn.Linear(self.hidden_dim * 4, 1)
        )
    
    def forward(self, query, passage):
        x = torch.cat([query, passage], dim=-1)
        x = self.feed_forward(x)
        return x

In [4]:
train = dataset['train']

In [5]:
queries = train['query']
passages = [item['passages']['passage_text'][0] for item in train]

In [6]:
queries_encodings = bi_encoder.encode(queries, convert_to_tensor=True)
passages_encodings = bi_encoder.encode(passages, convert_to_tensor=True)

In [7]:
import itertools

passages_negative_encodings = list(itertools.islice(itertools.cycle(passages_encodings), 1, len(passages_encodings) + 1))

In [8]:
training_examples = []

for i in range(len(queries_encodings)):
    training_examples.append((queries_encodings[i], passages_encodings[i], 1))
    training_examples.append((queries_encodings[i], passages_negative_encodings[i], -1))

In [9]:
import random
random.shuffle(training_examples)

In [11]:
model = Model(384).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

for epoch in range(1):
    total_loss = 0
    for i, (query, passage, label) in enumerate(training_examples):
        optimizer.zero_grad()
        output = model(query.cuda(), passage.cuda())
        loss = torch.nn.functional.margin_ranking_loss(output, torch.zeros_like(output).cuda(), torch.tensor([label]).cuda(), margin=1)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        log_interval = 1000
        if i % log_interval == 0:
            cur_loss = total_loss / log_interval
            print(f'epoch {epoch}, iter {i}, loss {cur_loss}')
            total_loss = 0

epoch 0, iter 0, loss 0.0009935027360916137
epoch 0, iter 1000, loss 0.989669047653675
epoch 0, iter 2000, loss 0.9552439688444138
epoch 0, iter 3000, loss 0.8340321342349053
epoch 0, iter 4000, loss 0.6368400468826294
epoch 0, iter 5000, loss 0.5128733741641045
epoch 0, iter 6000, loss 0.4759647963047028
epoch 0, iter 7000, loss 0.38384677821397784
epoch 0, iter 8000, loss 0.3528786128759384
epoch 0, iter 9000, loss 0.3324982855916023
epoch 0, iter 10000, loss 0.30596018832921984
epoch 0, iter 11000, loss 0.27760964566469193
epoch 0, iter 12000, loss 0.29567714655399324
epoch 0, iter 13000, loss 0.273961021900177
epoch 0, iter 14000, loss 0.2603491520881653
epoch 0, iter 15000, loss 0.23496199256181716
epoch 0, iter 16000, loss 0.29490037977695466
epoch 0, iter 17000, loss 0.22955643409490586


KeyboardInterrupt: 

In [12]:
test = dataset['test']
test_queries = test['query']
test_passages = [item['passages']['passage_text'][0] for item in test]

In [15]:
torch.save(model, '~/models/cross-embeddings.pt')

In [13]:
test_queries_encodings = bi_encoder.encode(test_queries, convert_to_tensor=True)
test_passages_encodings = bi_encoder.encode(test_passages, convert_to_tensor=True)

In [ ]:
for i in range(10):
    print(model(test_queries_encodings[i+0:i+1].cuda(), test_passages_encodings[i:i+1].cuda()))

tensor([-1.0759], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([2.0054], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([3.7806], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([2.2359], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([2.2724], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([2.9846], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([2.9599], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([3.1467], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([0.3648], device='cuda:0', grad_fn=<SelectBackward0>)
tensor([1.2762], device='cuda:0', grad_fn=<SelectBackward0>)


In [14]:
recall = 0
for i in range(200):
    print(i, end=' ')
    query = test[i]['query']
    query  = bi_encoder.encode(query, convert_to_tensor=True).cuda()
    passage = [test[i] for i in range(200)]
    passage = [x['passages'] for x in passage]
    passage = [x['passage_text'][0] for x in passage]
    passage = bi_encoder.encode(passage, convert_to_tensor=True).cuda()
    # print(passage)
    out = model(query.unsqueeze(0).repeat(200, 1), passage)
    recall += 1.0 if i in list(torch.topk(out, 3, dim=0).indices[:, 0]) else 0.0

print()
print(recall / 200)
# print()

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 
0.745


In [ ]:
query.unsqueeze(0).repeat(100, 1).shape

torch.Size([100, 384])

In [ ]:
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-2-v2', num_labels=1)

In [ ]:
recall = 0
for i in range(200):
    print(i, end=' ')
    out = cross_encoder.predict([[test[i]['query'], test[j]['passages']['passage_text'][0]] for j in range(200)])
    out = torch.from_numpy(out)
    recall += 1.0 if i in list(torch.topk(out, 3).indices) else 0.0
print()
print(recall/200)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 
0.92


In [ ]:
from sentence_transformers import util
recall = 0
for i in range(200):
    print(i, end=' ')
    test_passages = [test[j]['passages']['passage_text'][0] for j in range(200)]
    query_embedding = bi_encoder.encode(test[i]['query'], convert_to_tensor=True).cuda()
    passage_embeddings = bi_encoder.encode(test_passages, convert_to_tensor=True).cuda()
    cos_sim = torch.nn.functional.cosine_similarity(query_embedding.unsqueeze(0), passage_embeddings, dim=1)
    # print(torch.topk(cos_sim, 1).indices)
    recall += 1.0 if i in list(torch.topk(cos_sim, 3).indices) else 0.0
print()
print(recall/200)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 
0.955


In [ ]:
test[0]

{'answers': ['Yes,Augmentin comes in liquid form.'],
 'passages': {'is_selected': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
  'passage_text': ['1 Upload failed. 2  We are experiencing some problems, please try again. 3  You can only upload files of type PNG, JPG, or JPEG. 4  You can only upload files of type 3GP, 3GPP, MP4, MOV, AVI, MPG, MPEG, or RM. 5  You can only upload photos smaller than 5 MB. 6  You can only upload videos smaller than 600MB.',
   'Good dish for sharing, I don’t recommend does augmentin come in liquid form to eat on your own as you will get “ni” of does augmentin come in liquid form fast. 12 Hour does augmentin come in liquid form Braised Beef Short Ribs – $18. The braised beef short ribs was good. Very tender and juicy and I liked the potato pancake!',
   '| Best Cheaps🔥 | ☀☀☀ does augmentin come in liquid form ☀☀☀. Free shipping, quality, privacy, secure. does augmentin come in liquid form,Big Discounts No Prescription Required. Fully licensed,. Get started now!',
   'Am